In [1]:
%pip install transformers --upgrade
%pip install torch --upgrade

     |████████████████████████████████| 2.1MB 21.5MB/s 
     |████████████████████████████████| 870kB 50.4MB/s 
     |████████████████████████████████| 3.3MB 43.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=2bab98e2eba14647da2f8a334ffa982a47e75f2e4306d1fb3e07cffc5fba79d6
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses
Requirement already up-to-date: torch in /usr/local/lib/python3.7/dist-packages (1.8.1+cu101)


In [2]:
import torch
from torch import nn

from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from transformers import AutoTokenizer, AutoModel, AdamW
from sklearn.utils.class_weight import compute_class_weight
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
def clean_data(data):
    stop = stopwords.words('russian')
    data['message'] = data['message'].replace('/n', ' ')
    data['message'] = data['message'].map(lambda x: x.lower())
    data['message'] = data['message'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    return data

In [4]:
# Предзагрузка готовой модели ruBert с https://huggingface.co/
tokenizer = AutoTokenizer.from_pretrained("DeepPavlov/rubert-base-cased-sentence")
bert = AutoModel.from_pretrained("DeepPavlov/rubert-base-cased-sentence")

In [5]:
# Загрузка трейна
data = pd.read_csv("pulse_stage1_patch.csv")
data["label"] = data["label"].astype(int)
train_data = data.iloc[np.where(data.split == 'valid')[0], :].drop(['split'], axis=1)
val_data = data.iloc[np.where(data.split == 'train')[0], :].drop(['split'], axis=1)

# Загрузка теста
test_data = pd.read_csv('test.csv')

In [6]:
data['len'] = data['message'].str.len()
data.sort_values(by='len', ascending=False).drop(columns='len')

,message,label,split
18168,Спасибо большое Тинькофф инвестиции! Спасибо о...,0,train
13385,Акция хорошо выросла. Всем держателям этой акц...,0,valid
33706,"Да ситуация на самом деле отвратная, много зна...",0,train
7219,"2-3 недели, не больше. Я думаю до 1 Марта буде...",0,train
34392,А что же будет дальше? Ведь акциями не расплат...,0,valid
...,...,...,...
14579,Бред,0,valid
31305,Жопа,0,train
29541,бред,0,train
2497,Збс,1,train


In [7]:
train_data = clean_data(train_data)
val_data = clean_data(val_data)
#val_data['message'] = val_data['message'].replace('/n', ' ')
#val_data['message'] = val_data['message'].map(lambda x: x.lower())
#val_data['message'] = val_data['message'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
val_data.label = val_data.label.astype(int)
train_data.label = train_data.label.astype(int)
train_text = train_data.message
val_text = val_data.message
train_labels = train_data.label
val_labels = val_data.label

In [8]:
max_token_len = 40
tokens_train = tokenizer.batch_encode_plus(
    train_text.tolist(),
    max_length=max_token_len,
    pad_to_max_length=max_token_len,
    truncation=True,
    return_token_type_ids=False
)
tokens_val = tokenizer.batch_encode_plus(
    val_text.tolist(), 
    max_length=max_token_len,
    pad_to_max_length=max_token_len,
    truncation=True,
    return_token_type_ids=False
)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [9]:
train_seq = torch.tensor(tokens_train['input_ids'])
train_mask = torch.tensor(tokens_train['attention_mask'])
train_y = torch.tensor(train_labels.tolist())

# for validation set
val_seq = torch.tensor(tokens_val['input_ids'])
val_mask = torch.tensor(tokens_val['attention_mask'])
val_y = torch.tensor(val_labels.tolist())

In [10]:
#define a batch size
batch_size = 512

# wrap tensors
train_data = TensorDataset(train_seq, train_mask, train_y)

# sampler for sampling the data during training
train_sampler = RandomSampler(train_data)

# dataLoader for train set
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# wrap tensors
val_data = TensorDataset(val_seq, val_mask, val_y)

# sampler for sampling the data during training
val_sampler = SequentialSampler(val_data)

# dataLoader for validation set
val_dataloader = DataLoader(val_data, sampler = val_sampler, batch_size=batch_size)

#TRAIN PART

In [11]:
class BERT_Arch(nn.Module):

    def __init__(self, bert):
      
        super(BERT_Arch, self).__init__()

        self.bert = bert 

        # dropout layer
        self.dropout = nn.Dropout(0.3)

        # relu activation function
        self.relu =  nn.ReLU()

        # dense layer 1
        self.fc1 = nn.Linear(768, 512)

        # dense layer 2 (Output layer)
        self.fc2 = nn.Linear(512, 2)
        
        #softmax activation function
        self.softmax = nn.LogSoftmax(dim=1)

    #define the forward pass
    def forward(self, sent_id, mask):

        #pass the inputs to the model 
        _, cls_hs = self.bert(sent_id, attention_mask=mask, return_dict=False)
        x = self.fc1(cls_hs)

        x = self.relu(x)

        x = self.dropout(x)

        # output layer
        x = self.fc2(x)

        # apply softmax activation
        x = self.softmax(x)
        
        return x

In [12]:
# freeze all the parameters
for param in bert.parameters():
    param.requires_grad = False
model = BERT_Arch(bert)

In [13]:
#!g1.1
device = torch.device("cuda")

model = model.to(device)

In [14]:
# define the optimizer
optimizer = AdamW(model.parameters(), lr = 1e-3)

In [15]:
#compute the class weights
class_wts = compute_class_weight('balanced', np.unique(train_labels), train_labels)
class_wts = [0.7, 3]
print(f'class weights are: {class_wts}')
weights= torch.tensor(class_wts,dtype=torch.float)
weights = weights.to(device)

# loss function
cross_entropy  = nn.NLLLoss(weight=weights)

class weights are: [0.7, 3]


In [16]:
def train():
  
    model.train()
    total_loss, total_accuracy = 0, 0
  
    # empty list to save model predictions
    total_preds=[]
  
    # iterate over batches
    for step,batch in enumerate(train_dataloader):
    
    # progress update after every 50 batches.
        if step % 50 == 0 and not step == 0:
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(train_dataloader)))

        # push the batch to gpu
        batch = [r.to(device) for r in batch]
 
        sent_id, mask, labels = batch

        # clear previously calculated gradients 
        model.zero_grad()        

        # get model predictions for the current batch
        preds = model(sent_id, mask)

        # compute the loss between actual and predicted values
        loss = cross_entropy(preds, labels)

        # add on to the total loss
        total_loss = total_loss + loss.item()

        # backward pass to calculate the gradients
        loss.backward()

        # clip the the gradients to 1.0. It helps in preventing the exploding gradient problem
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # update parameters
        optimizer.step()

        # model predictions are stored on GPU. So, push it to CPU
        preds=preds.detach().cpu().numpy()

        # append the model predictions
        total_preds.append(preds)

    # compute the training loss of the epoch
    avg_loss = total_loss / len(train_dataloader)

    # predictions are in the form of (no. of batches, size of batch, no. of classes).
    # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)
    #returns the loss and predictions
    return avg_loss, total_preds

In [17]:
def evaluate():
  
    print("\nEvaluating...")
  
    # deactivate dropout layers
    model.eval()

    total_loss, total_accuracy = 0, 0
  
    # empty list to save the model predictions
    total_preds = []
    total_labels = []

    # iterate over batches
    for step,batch in enumerate(val_dataloader):
    
        # Progress update every 50 batches.
        if step % 50 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.'.format(step, len(val_dataloader)))

        # push the batch to gpu
        batch = [t.to(device) for t in batch]

        sent_id, mask, labels = batch
        
        # deactivate autograd
        with torch.no_grad():
      
            # model predictions
            preds = model(sent_id, mask)

            # compute the validation loss between actual and predicted values
            loss = cross_entropy(preds, labels)

            total_loss = total_loss + loss.item()

            preds = preds.detach().cpu().numpy()
            labels = labels.detach().cpu().numpy()
    
            total_labels.append(labels)
            total_preds.append(preds)

    # compute the validation loss of the epoch
    avg_loss = total_loss / len(val_dataloader)

    # reshape the predictions in form of (number of samples, no. of classes)
    total_preds  = np.concatenate(total_preds, axis=0)
    total_preds = np.argmax(total_preds, axis = 1)
    
    total_labels  = np.concatenate(total_labels, axis=0)
    print(classification_report(total_labels, total_preds))

    return avg_loss, total_preds

In [18]:
epochs = 10
best_valid_loss = float('inf')

#for each epoch
for epoch in range(epochs):
     
    print('\n Epoch {:} / {:}'.format(epoch + 1, epochs))
    
    #train model
    train_loss, _ = train()
    
    #evaluate model
    valid_loss, _ = evaluate()
    
    #save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'toxic_bert_wts.pt')
    
    print(f'\nTraining Loss: {train_loss:.3f}')
    print(f'Validation Loss: {valid_loss:.3f}')


 Epoch 1 / 10

Evaluating...
  Batch    50  of     55.


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      1.00      0.98     26788
           1       0.00      0.00      0.00       913

    accuracy                           0.97     27701
   macro avg       0.48      0.50      0.49     27701
weighted avg       0.94      0.97      0.95     27701


Training Loss: 0.370
Validation Loss: 0.161

 Epoch 2 / 10

Evaluating...
  Batch    50  of     55.
              precision    recall  f1-score   support

           0       0.98      0.98      0.98     26788
           1       0.35      0.35      0.35       913

    accuracy                           0.96     27701
   macro avg       0.67      0.66      0.66     27701
weighted avg       0.96      0.96      0.96     27701


Training Loss: 0.233
Validation Loss: 0.129

 Epoch 3 / 10

Evaluating...
  Batch    50  of     55.
              precision    recall  f1-score   support

           0       0.98      0.97      0.97     26788
           1       0.33      0.48  

In [19]:
test_text = test_data.text
tokens_test = tokenizer.batch_encode_plus(
    test_text.tolist(), 
    max_length=40,
    pad_to_max_length=40,
    truncation=True,
    return_token_type_ids=False
)
test_seq = torch.tensor(tokens_test['input_ids'])
test_mask = torch.tensor(tokens_test['attention_mask'])

with torch.no_grad():
    preds = model(test_seq.to(device), test_mask.to(device))
    preds = preds.detach().cpu().numpy()
preds = np.argmax(preds, axis = 1)


dataframe_pred = pd.DataFrame(data = test_data.text)
dataframe_pred.columns = ["text"]
dataframe_pred["toxic"] = preds
dataframe_pred.to_csv("submittion.csv", index=False)


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
